In [3]:
from utils import CanvasSession
from selenium.webdriver.common.by import By
from canvasapi import Canvas
from config import *
import time
import pickle

# import beautifulsoup
from bs4 import BeautifulSoup

canvas = Canvas(CANVAS_URL, CANVAS_TOKEN)

In [9]:
# Get submissions and grade centre urls
submissions = [x for x in canvas.get_course(course_id).get_assignment(assignment_id).get_submissions(include=["user", "submission_comments"]) if x.workflow_state != "unsubmitted"]
urls = [f"{CANVAS_URL}/courses/{course_id}/gradebook/speed_grader?assignment_id={assignment_id}&student_id={x.user_id}" for x in submissions]

In [13]:
# Login to Canvas
session = CanvasSession()

wait ...
Verification code for DUO
4233
You have 1 seconds to enter your verification code..
Login Success.


In [14]:
session

In [18]:
# This function gets the annotations
def get_annotations(session, url):

    session.browser.get(url)
    print(session.browser.current_url)
    wait = 0
    while True:
        print("Loading ...")
        try:
            session.browser.switch_to.frame('speedgrader_iframe')
            print("Switching to iframe")
            time.sleep(5)

            annotations = []

            html_string = session.browser.page_source
            soup = BeautifulSoup(html_string, 'html.parser')
            annotation_authors = soup.find_all('div', {'class': 'ScreenreaderAnnotation-author'})
            annotation_comments = soup.find_all('div', {'class': 'ScreenreaderAnnotation-root-comment'})

            for author, annotation in zip(annotation_authors, annotation_comments):
                comment = {
                    "author": author.get_text().split(":")[-1],
                    "comment": annotation.get_text().split(":")[-1],
                    "type": "annotation"
                }

                annotations.append(comment)
                  
            session.browser.switch_to.default_content()
            print("Switching to default content")
            break
        except:
            if wait > 20:
                annotations = []
                break
            else:
                time.sleep(2)
                wait += 2
                continue

    return annotations    

In [19]:
urls[10]

'https://canvas.liverpool.ac.uk/courses/69023/gradebook/speed_grader?assignment_id=256081&student_id=55719'

In [22]:
test = get_annotations(session, urls[10])

https://canvas.liverpool.ac.uk/courses/69023/gradebook/speed_grader?assignment_id=256081&student_id=55719
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Loading ...
Switching to iframe
Switching to default content


In [23]:
test

[{'author': 'Antonius Plagge',
  'comment': 'very good, important points',
  'type': 'annotation'},
 {'author': 'Antonius Plagge',
  'comment': 'Good introduction. You could have more clearly stated you personal opinion on the question, since this is an essay, which reflects a personal view or opinion that your are arguing for in the following sections (in contrast to a balanced scientific report that simply summaries data). ',
  'type': 'annotation'},
 {'author': 'Antonius Plagge',
  'comment': 'Shreiner et al., 2015 (more than 2 authors)',
  'type': 'annotation'},
 {'author': 'Antonius Plagge',
  'comment': 'well written section',
  'type': 'annotation'},
 {'author': 'Antonius Plagge',
  'comment': 'there should be a reference at the end of this sentence',
  'type': 'annotation'},
 {'author': 'Antonius Plagge',
  'comment': 'difficult to read the small print / low image resolution',
  'type': 'annotation'},
 {'author': 'Antonius Plagge',
  'comment': 'yes, important point here. ',
  

In [12]:
from tqdm import tqdm
# Loop through submissions and get comments
with open("output.pkl", "rb") as f:
    data = pickle.load(f)

comments = []

for submission, url in tqdm(zip(submissions[2:3], urls[2:3]), total=len(submissions[2:])):
    if submission.user_id in data:
        continue
        #print(url)
        #print(data[submission.user_id])
        #print(f"Skipping {submission.user['sortable_name']}")
    else:
        session.browser.get(url)
        print(session.browser.current_url)
        print(submission.user['sortable_name'])
        wait = 0
        while True:
            try:
                session.browser.switch_to.frame('speedgrader_iframe')
                time.sleep(15)
                comments = get_comments(session)
                session.browser.switch_to.default_content()
                break
            except:
                if wait > 20:
                    comments = []
                    break
                else:
                    time.sleep(2)
                    wait += 2
                    continue

        try:
            submission_comments = [x.comment for x in submission.submission_comments]
        except:
            submission_comments = []

        data[submission.user_id] = {
            "sortable_name": submission.user["sortable_name"],
            "score": submission.score,
            "annotations": comments,
            "submission_comments": submission_comments
        }
        print(data[submission.user_id])


  0%|          | 0/425 [00:00<?, ?it/s]

https://canvas.liverpool.ac.uk/courses/71993/gradebook/speed_grader?assignment_id=261378&student_id=54716
McCormack, Carl


  2%|▏         | 10/425 [00:42<29:14,  4.23s/it]

{'sortable_name': 'McCormack, Carl', 'score': 10.0, 'annotations': [], 'submission_comments': []}
https://canvas.liverpool.ac.uk/courses/71993/gradebook/speed_grader?assignment_id=261378&student_id=78993
Are, Amaal


  8%|▊         | 34/425 [01:24<15:06,  2.32s/it]

...

{'sortable_name': 'Khan, Amy', 'score': 48.0, 'annotations': [], 'submission_comments': []}
https://canvas.liverpool.ac.uk/courses/71993/gradebook/speed_grader?assignment_id=261378&student_id=143863
Yao, Jintian


 99%|█████████▉| 420/425 [12:15<00:05,  1.14s/it]

{'sortable_name': 'Yao, Jintian', 'score': 22.0, 'annotations': [], 'submission_comments': []}
https://canvas.liverpool.ac.uk/courses/71993/gradebook/speed_grader?assignment_id=261378&student_id=147948
student, Test


100%|██████████| 425/425 [12:41<00:00,  1.79s/it]

{'sortable_name': 'student, Test', 'score': 0.0, 'annotations': [], 'submission_comments': []}


In [88]:
# read the output.pkl file
with open('output.pkl', 'rb') as f:
    data = pickle.load(f)

In [89]:
data

{7523: {'sortable_name': 'Campbell, David',
  'score': 11.0,
  'annotations': [{'author': 'Sophie Lyth',
    'comment': '0/5',
    'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '0/5', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '2/5', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '1/5', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '2/5', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '1/4', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '2/10', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '2/4', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '1/6', 'type': 'annotation'}],
  'submission_comments': []},
 9658: {'sortable_name': 'Cleary, Jack',
  'score': 30.0,
  'annotations': [{'author': 'Sophie Lyth',
    'comment': '4/5',
    'type': 'annotation'},
   {'author': 'Sophie Lyth', 'comment': '4/5', 'type': 'annotation'},
   {'author': 'Sophie Lyth', 'com

In [91]:
url = session.browser.get("https://canvas.liverpool.ac.uk/courses/71993/gradebook/speed_grader?assignment_id=261378&student_id=9658")
time.sleep(10)
session.browser.switch_to.frame('speedgrader_iframe')
time.sleep(10)
comments = get_comments(session)
for comment in comments:
    print(comment)
session.browser.switch_to.default_content()

{'author': 'Sophie Lyth', 'comment': '4/5', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '4/5', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '5/5', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '3/5', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '3/5', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '1/10', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '2/4', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '2/3', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '3/4', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '3/6', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '0/4', 'type': 'annotation'}
{'author': 'Sophie Lyth', 'comment': '0/6', 'type': 'annotation'}
